#  les colonnes catégoriques et numériques

categorical and numerical features

In [ ]:
import pandas as pd
import numpy as np

# colonnes catégoriques ou numériques

In [ ]:
df_adult = pd.read_csv('adult.csv')
# fichier un peu trafiqué qui peut contenir des différences avec l'original

In [ ]:
df_adult.head(2)

## sélectionnées en `pandas`

In [ ]:
df_adult.dtypes

les colonnes `object` sont les colonnes non-numériques  
donc catégorielles

In [ ]:
df_adult.select_dtypes(include=object).head(2)

les colonnes numériques

In [ ]:
df_adult.select_dtypes(include='number').head(2)

## sélectionnées en `sklearn`

In [ ]:
from sklearn.compose import make_column_selector as selector

In [ ]:
cat_sel = selector(dtype_include=object) # en sklearn (qui utilise select_dtypes de pandas)
cat_columns = cat_sel(df_adult)
cat_columns

In [ ]:
num_sel = selector(dtype_include='number')
num_sel(df_adult)

## colonnes catégorielles  `int` ou `float`

besoin de comprendre ce que contiennent les colonnes  
et la relation du contenu avec le type de la colonne en `pandas`  

(`float` peut apparaître lors de valeurs manquantes dans une colonne de `int`)

on crée une dataframe avec une colonne de genre ($1$ pour masculin et $2$ pour féminin) comportant des valeurs manquantes

In [ ]:
df_genre = pd.DataFrame(np.random.choice([1, 2, np.nan], size=100), columns=['genre'])
df_genre.dtypes

vous remarquez le type **`float`**
- `np.nan` est de type `float`  
- `pandas` n'a pas (encore)  d'équivalent à `np.nan` pour les `int`  
- donc cette colonne `sexe` est de type `float` et pas `int`

In [ ]:
df_genre.head(2)

**mais** cette colonne
- n'est pas une variable réelle (même si elle est de type `float`)
- n'est pas une variables ordonnée ici $1 \not< 2$
- elle ne doit pas être traitée comme une colonne à valeurs réelle lors de son utilisation dans des modèles de prédictions  


on peut aussi s'en rendre compte en comptant les valeurs contenues dans cette colonne:

In [ ]:
df_genre['genre'].unique() # les valeurs

In [ ]:
df_genre.value_counts(dropna=False) # leur distribution

# passage en type `category`

## catégories non-ordonnées en `pandas`

In [ ]:
df_adult_pd = df_adult[cat_columns].copy()
df_adult_pd.dtypes # des types object (pointeur de 64 bits)

In [ ]:
df_adult_pd = df_adult_pd.astype('category')

In [ ]:
df_adult_pd.dtypes

In [ ]:
df_adult_pd['marital-status'][0:2]

pour avoir les codes:

In [ ]:
df_adult_pd['marital-status'].cat.codes

In [ ]:
df_adult_pd['marital-status'].cat.categories

In [ ]:
df_adult_pd['marital-status'].cat.codes.value_counts()

Ici dans `marital-status`
- `Divorced` est à $0$
- `Widowed` est à $6$

ça n'a bien sûr aucun sens pour des calcul: ces valeurs ne doivent pas être utilisées lors de prédictions !

## catégories ordonnées en `pandas`

`ordinal encoding`

Wikipedia

*Ordinal data is a categorical, statistical data type where the variables have natural, ordered categories and the distances between the categories are not known.*

Attention: ne pas attribuer ces codages à une variable catégorielle non ordonnée

par contre, pour des catégories ordonnées, ces codes ont du sens

e.g. `petit` à $0$ , `moyen` à $1$ et `grand` à $2$

In [ ]:
df_size = pd.DataFrame(np.random.choice(['petit', 'moyen', 'grand'], size=100),
                       columns=['size'])
df_size['size'].value_counts()

In [ ]:
from pandas.api.types import CategoricalDtype

In [ ]:
cat_type = CategoricalDtype(categories=['petit', 'moyen', 'grand'], ordered=True)
                                       # on donne l'ordre des classes dans la catégorie

on met ce type comme type de la colonne  
un type de variable ordinale où 'petit' < 'moyen' < 'grand'

In [ ]:
df_size['size'] = df_size['size'].astype(cat_type)
# df['size'].value_counts()

## one-hot-encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

quand les catégories ne sont pas ordonnées, pour utiliser ces colonnes dans des calculs de prédiction  
on peut utiliser un one-hot-encoding

pour une colonne `color` avec les catégories `green` , `white` et `red`
- on crée autant de colonne que de valeurs de catégories  
(ici $3$ colonnes) `color_green` `color_white` et `color_red`


- et pour chaque observation
   - on met `1` dans la colonne qui correspond à sa catégorie  
   e.g. si l'observation est `red` dans `color`, elle sera `1` dans `color_red`
   - et $0$ dans les autres colonnes  
   e.g. si l'observation est `red` dans `color`, elle sera `0` dans `color_white` et dans `color_green`
   
   
les nouvelles colonnes n'ont plus aucun lien alors qu'elles représentent pourtant la même information  
(on perd de l'information)

In [ ]:
encoder = OneHotEncoder(sparse=False, # c'est par construction très très parse donc il faut mettre sparse à True
                                      # et PAS à False
                                      # mais là on veut montrer ce qui se passe et pas calculer (donc on le fait)
                        handle_unknown="ignore", # pour éviter le problème d'une catégorie
                                                 # présente dans le jeux de test
                                                 # et pas dans le jeu d'apprentissage
                                                 # on lui demande d'ignorer ces erreurs
                                                 # et de ne pas lancer une exception
                        # dtype=int 
                       )

new_cols = encoder.fit_transform(df_adult[cat_columns])

# on part d'un tableau de 9 colonnes

print(len(cat_columns))


# on récupère un tableau numpy de 104 colonnes pour les 9 initiales

print(new_cols.shape[1])

# les colonnes sont pleines de vide ...

new_cols

In [ ]:
# l'encoder a généré les 104 noms de colonnes
feature_names = encoder.get_feature_names_out(input_features=cat_columns)
feature_names.shape

In [ ]:
# on peut en faire une dataframe (si besoin)
pd.DataFrame(new_cols, columns=feature_names).head(2)

## prédiction du salaire par regression

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('adult.csv')
cat_columns = ['workclass', 'education', 'marital-status', 'occupation',
               'relationship', 'race', 'sex', 'native-country']
num_columns = ['age', 'capital-gain', 'capital-loss', 'hours-per-week',]

### only one-hot encoded columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

In [ ]:
model_hot = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"),
    LogisticRegression(max_iter=500)
)

X = df[cat_columns]
y = df['class'].astype('category')

In [ ]:
cv_results_hot = cross_validate(model_hot, X, y)
cv_results_hot

In [ ]:
scores_hot = cv_results_hot["test_score"]
print(f"The accuracy is: {scores_hot.mean():.3f} ± {scores_hot.std():.3f}")

### only numeric columns

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler

In [ ]:
model_num = make_pipeline(
    StandardScaler(),
    LogisticRegression()
)

X = df[num_columns]
y = df['class'].astype('category')#.cat.codes

In [ ]:
cv_results_num = cross_validate(model_num, X, y)
cv_results_num

In [ ]:
scores_num = cv_results_num["test_score"]
print(f"The accuracy is: {scores_num.mean():.3f} ± {scores_num.std():.3f}")

### with all columns

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [ ]:
ct = ColumnTransformer(
    [('std_scaler', StandardScaler(), num_columns),
     ('ont_hot_encoding',
      OneHotEncoder(handle_unknown="ignore", sparse=True), # try False to see the difference in fit times
      cat_columns),
    ])

In [ ]:
model_all = make_pipeline(
    ct,
    LogisticRegression(max_iter=500)
)
X = df[cat_columns+num_columns]
y = df['class']

In [ ]:
cv_results_all = cross_validate(model_all, X, y)
cv_results_all

In [ ]:
scores_all = cv_results_all["test_score"]
print(f"The accuracy is: {scores_all.mean():.3f} ± {scores_all.std():.3f}")

### colonne déjà encodée

la dataframe originale contient une colonne qui est l'encodage d'une autre

In [ ]:
df = pd.read_csv('adult.data', header=None)

In [ ]:
df.head(2)

ce sont les codes ordonnés de la colonne `education`

on peut le voir en calculant des fréquences

In [ ]:
col1 = 3 # colonne des noms de education
val1 = ' Bachelors'  # remarquez le ' ' en début de str...

col2 = 4 # colonne des codes ordonnés de education
val2 = 13

np.sum((df[col1] == val1)  &  (df[col2] == val2)) == np.sum(df[col1] == val1) == np.sum(df[col2] == val2)

# toutes observations qui ont val1 dans la colonne col1
#                         ont val2 dans la colonne col2
# et ce nombre est le nombre total des valeurs dans chacune des colonnes

on utilise `pd.crosstab` qui calcule cette table des fréquences des valeurs

In [ ]:
pd.crosstab(df[4], df[3])

on essaie sur les colonnes education et workclass

In [ ]:
pd.crosstab(df[1], df[3])
# c'est très mélangé pas de lien direct évident

exercice: mettre cet encodage pour la colonne `education` à la place d'un one-hot-encoding de `education` et relancer l'apprentissage

## jeu de test

le fichier `adult.test` contient un jeu de test  
attention sa première ligne est un commentaire qu'il faut skipper  
et il n'y a pas de noms de colonnes

In [ ]:
df_test = pd.read_csv('adult.test', skiprows=1, header=None)
df_test

exercice: utiliser ce jeu de données de test pour calculer les score de généralisation des différents prédicteurs que nous avons entraînés dans ce notebook